### 4. Γεωμετρική Αναζήτηση

In [233]:
class InternalNode:
    def __init__(self, x_mid):
        self.x_mid = x_mid
        self.left = None
        self.right = None
        self.assoc = None
        self.S = None

class LeafNode:
    def __init__(self, point):
        self.point = point
        self.assoc = None

def split_by_x(Py, x_mid):
    Py_left = []
    Py_right = []
    
    for p in Py:
        if p[0] <= x_mid:
            Py_left.append(p)
        else:
            Py_right.append(p)
    
    return np.array(Py_left), np.array(Py_right)

def build_2d_range_tree(points):
    animation_frames = []
    def build_tree(P):
        # print(P["len"])
        if P["len"] == 0:
            return None
        Tassoc = build_bst(P["y"])
        if P["len"] == 1:
            v = LeafNode(point=P["x"][0])
            v.assoc = Tassoc
            return v
        else:
            # P.sort(key=lambda p: p[0])  # Sort by x-coordinate
            Px = P["x"]
            Py = P["y"]
            mid = P["len"] // 2
            print(mid)
            x_mid = Px[mid-1][0]
            P_left = {}
            P_right = {}
            P_left["x"] = Px[:mid]
            P_right["x"] = Px[mid:]
            P_left["y"], P_right["y"] = split_by_x(Py, x_mid)
            P_left["len"] = len(P_left["x"])
            P_right["len"] = len(P_right["x"])
            v = InternalNode(x_mid=x_mid)
            v.S = Px
            v.left = build_tree(P_left)
            v.right = build_tree(P_right)
            v.assoc = Tassoc
            animation_frames.append(v)
            return v

    def build_bst(P):
        if not len(P):
            return None
        mid = len(P) // 2
        if len(P) == 1:
            return LeafNode(point=P[0])
        node = InternalNode(x_mid=P[mid-1][1])  # Using y-coordinate as x_mid for the associated structure
        node.left = build_bst(P[:mid])
        node.right = build_bst(P[mid:])
        node.S = P
        return node

    return build_tree(points)

# Helper function to print the tree (for debugging)
def print_tree(node, level=0):
    if isinstance(node, LeafNode):
        print("  " * level + f"Leaf: point={node.point}")
    elif isinstance(node, InternalNode):
        print("  " * level + f"Internal: x_mid={node.x_mid}")
        print("  " * level + f"Assoc:")
        print_tree(node.assoc, level+3)
        print("  " * level + "Left:")
        print_tree(node.left, level + 1)
        print("  " * level + "Right:")
        print_tree(node.right, level + 1)

# Example usage
import numpy as np
rng = np.random.default_rng(seed=42)
n = 8
x = [i for i in range(n)]
y = [i for i in range(n)]
P = np.array([rng.permutation(x), rng.permutation(y)]).T
points = np.array(
    [
        (86, 340),
        (163, 135),
        (234, 292),
        (293, 440),
        (373, 206),
        (427, 63),
        (482, 193),
        (614, 68)
    ]
)
# print(P)
# return    
Px = np.array(sorted(points, key=lambda p: p[0]))
Py = np.array(sorted(points, key=lambda p: p[1]))
mid=len(Px)//2
Px_left = Px[:mid+1]
Py_left = Py[:mid+1]
# print(Px_left)
# print(Py_left)

P = {
    "x" : Px,
    "y" : Py,
    "len": len(P)
}
root = build_2d_range_tree(P)
print_tree(root)

4
2
1
1
2
1
1
Internal: x_mid=293
Assoc:
      Internal: x_mid=193
      Assoc:
      Left:
        Internal: x_mid=68
        Assoc:
        Left:
          Internal: x_mid=63
          Assoc:
          Left:
            Leaf: point=[427  63]
          Right:
            Leaf: point=[614  68]
        Right:
          Internal: x_mid=135
          Assoc:
          Left:
            Leaf: point=[163 135]
          Right:
            Leaf: point=[482 193]
      Right:
        Internal: x_mid=292
        Assoc:
        Left:
          Internal: x_mid=206
          Assoc:
          Left:
            Leaf: point=[373 206]
          Right:
            Leaf: point=[234 292]
        Right:
          Internal: x_mid=340
          Assoc:
          Left:
            Leaf: point=[ 86 340]
          Right:
            Leaf: point=[293 440]
Left:
  Internal: x_mid=163
  Assoc:
        Internal: x_mid=292
        Assoc:
        Left:
          Internal: x_mid=135
          Assoc:
          Left:
    

In [234]:
def split_node(node, x_range):
    if node is None:
        return None
    if isinstance(node, LeafNode):
        return node
    else:
        if node.x_mid >= x_range[0] and node.x_mid <= x_range[1]:
            return node
        elif node.x_mid < x_range[0]:
            return split_node(node.right, x_range)
        elif  node.x_mid > x_range[1]:
            return split_node(node.left, x_range)
def range_query(root, x_range, y_range):
    report = []
    def query_x(node, x_range):
        if node is None:
            return []
        
        if isinstance(node, LeafNode):
            if x_range[0] <= node.point[0] <= x_range[1] and y_range[0] <= node.point[1] <= y_range[1]:
                return [node.point]
            return []

        if node.x_mid <= x_range[0]:
            return query_x(node.right, x_range)
        elif node.x_mid > x_range[1]:
            return query_x(node.left, x_range)
        else:
            left_points = filter_y_range(node.left.S, y_range) if np.all(node.left.S) else query_x(node.left, x_range)
            right_points = query_x(node.right, x_range)
            return left_points + right_points

    def report_subtree(node):
        if isinstance(node, LeafNode):
            report.append(node.point)
        else:
            report += node.S
    def range_query_1d(node, r):
        node = split_node(node, r)
        if isinstance(node, LeafNode):
            report.append(node.point)
        else:
            v = node.left
            while not isinstance(v, LeafNode):
                if r[0] <= v.x_mid:
                    report_subtree(v.right)
                    v = v.left
                else:
                    v = v.right
    node = split_node(root, x_range)
    # print(node.x_mid)
    # return []
    if isinstance(node, LeafNode):
        if node.point[1] in y_range:
            report.append(node.point)
    else:
        v = node.left
        while not isinstance(v, LeafNode):
            if x_range[0] <= v.x_mid:
                range_query_1d(v.right.assoc, y_range)
                v = v.left
            else:
                v = v.right
        if v.point[0] >= x_range[0] and v.point[0] <= x_range[1] and v.point[1] >=  y_range[0] and v.point[1] <= y_range[1]:
            report.append(v.point)
        v = node.right
        while not isinstance(v, LeafNode):
            if x_range[1] >= v.x_mid:
                range_query_1d(v.left.assoc, y_range)
                v = v.right
            else:
                v = v.left
        if v.point[0] >= x_range[0] and v.point[0] <= x_range[1] and v.point[1] >=  y_range[0] and v.point[1] <= y_range[1]:
            report.append(v.point)
    # print(report)
    return report

# Helper function to print the results
def print_query_results(results):
    print(f"Found {len(results)} points in the given range:")
    for point in results:
        print(f"  {point}")

# Example usage
def example_query(root):
    x_range = (250, 500)
    y_range = (150, 350)
    results = range_query(root, x_range, y_range)
    print(f"Query range: x={x_range}, y={y_range}")
    print_query_results(results)

# To use this, you would call:
example_query(root)

Query range: x=(250, 500), y=(150, 350)
Found 2 points in the given range:
  [373 206]
  [482 193]
